In [1]:
import pandas as pd

In [2]:
def TPSL(xs):
    '''
    Expects:
      - LongTP
      - LongSL

    Creates:
      - LongTPSteps
      - LongSLSteps
    '''
    N = 100

    shadow = pd.DataFrame(index=xs.index)
    shadow['SLHit'] = False
    shadow['U1Hit'] = False
    xs['LongSLSteps'] = N
    xs['LongTPSteps'] = N

    for i in range(1, N+1):
        shadow['SLHit'] |= (xs.LongSL >= xs.Low.shift(-i))
        xs.loc[shadow['SLHit'], 'LongSLSteps'] = xs.LongSLSteps.clip(upper=i)

        shadow['U1Hit'] |= (xs.LongTP <= xs.High.shift(-i))
        xs.loc[shadow.U1Hit, 'LongTPSteps'] = xs.LongTPSteps.clip(upper=i)

    # Loss.
    xs['PNL'] = 1 / (xs.LongSL - xs.Close)

    # Profit.
    mask = xs.LongTPSteps == xs[['LongSLSteps', 'LongTPSteps']].min(axis=1)
    xs.loc[mask, 'PNL'] = 1 / (xs.LongTP - xs.Close)

    return xs